<a href="https://colab.research.google.com/github/Gorik27/MD_MC_course/blob/main/Task5_MDMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install cmake build-essential git ccache

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.12).
The following additional packages will be installed:
  libhiredis0.14
Suggested packages:
  distcc | icecc
The following NEW packages will be installed:
  ccache libhiredis0.14
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 528 kB of archives.
After this operation, 1,469 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhiredis0.14 amd64 0.14.1-2 [32.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ccache amd64 4.5.1-1 [495 kB]
Fetched 528 kB in 1s (552 kB/s)
Selecting previously unselected package libhiredis0.14:amd64.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .

In [2]:
!rm -rf lammps
!git clone -b stable --depth=1 https://github.com/lammps/lammps.git lammps

Cloning into 'lammps'...
remote: Enumerating objects: 14075, done.
remote: Counting objects: 100% (14075/14075), done.
remote: Compressing objects: 100% (10172/10172), done.
remote: Total 14075 (delta 4792), reused 7779 (delta 3671), pack-reused 0 (from 0)
Receiving objects: 100% (14075/14075), 128.79 MiB | 17.98 MiB/s, done.
Resolving deltas: 100% (4792/4792), done.
Updating files: 100% (13422/13422), done.


In [3]:
!rm -rf build
!mkdir build
!cd build; cmake -C ../lammps/cmake/presets/basic.cmake -D CMAKE_INSTALL_PREFIX=/usr -D CMAKE_CXX_COMPILER_LAUNCHER=ccache -D BUILD_SHARED_LIBS=on -D LAMMPS_EXCEPTIONS=on -D PKG_PYTHON=on -D PKG_MC=on  ../lammps/cmake
!cd build; make

loading initial cache file ../lammps/cmake/presets/basic.cmake
CMake Deprecation Warning at CMakeLists.txt:18 (cmake_policy):
  The OLD behavior for policy CMP0109 will be removed from a future version
  of CMake.

  The cmake-policies(7) manual explains that the OLD behaviors of all
  policies are deprecated and that a policy should be set to OLD only under
  specific short-term circumstances.  Projects should be ported to the NEW
  behavior and not rely on setting a policy to OLD.


-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Appending /usr/local/cuda/lib64/stubs to CMAKE_LIBRARY_PATH: /usr/local/cuda/lib64/stubs
-- Running check for auto-generated files from make-based build system
-- Found MPI_CXX: /usr/lib/x86_64-li

In [4]:
!cd build; make install

[  0%] Built target angle.h
[  0%] Built target kspace.h
[  0%] Built target lammps.h
[  0%] Built target lattice.h
[  0%] Built target library.h
[  0%] Built target lmppython.h
[  0%] Built target lmptype.h
[  0%] Built target memory.h
[  0%] Built target modify.h
[  0%] Built target neighbor.h
[  1%] Built target neigh_list.h
[  1%] Built target fmt_core.h
[  1%] Built target fmt_format.h
[  1%] Built target timer.h
[  1%] Built target universe.h
[  1%] Built target update.h
[  1%] Built target utils.h
[  2%] Built target variable.h
[  2%] Built target command.h
[  2%] Built target compute.h
[  2%] Built target dihedral.h
[  3%] Built target domain.h
[  3%] Built target atom.h
[  3%] Built target bond.h
[  3%] Built target citeme.h
[  3%] Built target comm.h
[  3%] Built target output.h
[  3%] Built target pair.h
[  3%] Built target platform.h
[  3%] Built target pointers.h
[  4%] Built target region.h
[  4%] Built target group.h
[  5%] Built target improper.h
[  5%] Built target inp

Writing LAMMPS input file
* LAMMPS manual https://www.afs.enea.it/software/lammps/doc17/html/Section_commands.html
* Interatomic potentials https://www.ctcms.nist.gov/potentials/

In [5]:
%%writefile in.create
######################################
# INITIALIZATION
clear
units metal
dimension 3
boundary p p p
atom_style atomic
atom_modify map array

######################################
# ATOM DEFINITION
lattice bcc 2.87
region box block 0 1 0 1 0 1 units lattice
create_box 2 box
create_atoms 1 box
replicate 20 20 20

timestep 0.001
######################################
# DEFINE INTERATOMIC POTENTIAL
pair_style eam/cd
pair_coeff * * FeCr.cdeam.txt Fe Cr
neighbor 2.0 bin
neigh_modify delay 10 check yes

######################################
# DEFINE OUTPUT

reset_timestep 0
thermo 10
thermo_style custom step pe press

######################################
# RUN
fix 1 all box/relax iso 0.0 vmax 0.001
minimize 1e-12 1e-8 1000000 0
unfix 1
minimize 1e-12 1e-8 1000000 0

write_data crystall.dat

print "All done!"

Writing in.create


In [7]:
! lmp -in in.create

LAMMPS (29 Aug 2024 - Update 2)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 2.87 2.87 2.87
Created orthogonal box = (0 0 0) to (2.87 2.87 2.87)
  1 by 1 by 1 MPI processor grid
Created 2 atoms
  using lattice units in orthogonal box = (0 0 0) to (2.87 2.87 2.87)
  create_atoms CPU = 0.000 seconds
Replication is creating a 20x20x20 = 8000 times larger system...
  orthogonal box = (0 0 0) to (57.4 57.4 57.4)
  1 by 1 by 1 MPI processor grid
  16000 atoms
  replicate CPU = 0.002 seconds
Reading eam/alloy potential file FeCr.cdeam.txt with DATE: 2009-11-05
Reading potential file FeCr.cdeam.txt with DATE: 2009-11-05
Switching to 'neigh_modify every 1 delay 0 check yes' setting during minimization
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check

**Task 5: MCMD simulation**


---



1.  Create a Fe crystal model with at least $20\times 20\times 20$ unit cells.
2. Simulate the FeCr alloy in the semi-grand canonical ensemble using command ```fix ID group-ID sgcmc every_nsteps swap_fraction temperature deltamu ...``` (see https://vcsgc-lammps.materialsmodeling.org/usage.html#syntax for details). It is convenient to use ```every_nsteps = 10``` and ```swap_fraction = 1.0```, but this is not necessary. Use the NPT ensemble to integrate the equations of motion.
3. Find the number of steps (MDMC) sufficient for convergence (after convergence concentration oscillates but shows no trend)
3. Find the dependence of the Cr concentration on $\Delta \mu$ (```deltamu```). The relevant range is approximately from $0.1$ to $-0.7 eV$. (**Hint**: 20 points are sufficient)

To automate the workflow, you can use Python. There are many ways to do so. The PYTHON package in LAMMPS, the Python library ASE, and the subprocess Python module are a few examples. I recommend using the subprocess module.

In [ ]:
from subprocess import Popen, PIPE
import time
import numpy as np


def lammps(task, nonverbose=True):
    exitflag = False
    error_msg = ''
    with Popen(task.split(), stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        time.sleep(0.1)
        for line in p.stdout:
            if "thermo output" in line:
                thermo = (line.replace('thermo output ', '')).replace('\n', '')
            elif "All done" in line:
                exitflag = True
            elif "ERROR" in line:
                error_msg = line.replace('ERROR: ', '')
            if nonverbose:
                if '!' in line:
                    print(line.replace('!', ''), end='')
            else:
                print(line, end='')

        if not exitflag:
            raise ValueError(f'Error in LAMMPS: {error_msg}')

script = '<script_name>'
mu_list = np.linspace(0.1, -0.7, num=20)
for mu in mu_list:
  task = f'lmp -in {script} -var mu {mu}'
  lammps(task)

where LAMMPS script includes the following variable: ```variable mu index 0``` (0 can be replaced by any value since the actual value will be replaced by ```-var mu <mu_value>``` argument)